In [64]:
import pandas as pd


In [65]:
pd.__version__

'2.2.3'

In [66]:
jan23 = pd.read_parquet("yellow_tripdata_2023-01.parquet")
feb23 = pd.read_parquet("yellow_tripdata_2023-02.parquet")

#### Q1: downloading the data

We'll use [the same NYC taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".
Download the data for January and February 2023.
Read the data for January. How many columns are there?

In [67]:
jan23.columns, len(jan23.columns)

(Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
        'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
        'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
        'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
        'total_amount', 'congestion_surcharge', 'airport_fee'],
       dtype='object'),
 19)

In [68]:
jan23.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


#### Q2: computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.
What's the standard deviation of the trips duration in January?

In [69]:
jan23["duration"] = jan23["tpep_dropoff_datetime"] - jan23["tpep_pickup_datetime"]
jan23["duration"] = jan23["duration"].dt.seconds / 60.0

In [70]:
jan23["duration"].std(ddof=1)

np.float64(41.62919110966266)

#### Q3: dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).
What fraction of the records left after you dropped the outliers?

In [71]:
rows_before = jan23.shape[0]

In [72]:
jan23 = jan23[(jan23["duration"] >= 1) & (jan23["duration"] <= 60)]

In [73]:
jan23.shape[0] / rows_before

0.9812212604417814

#### Q4: one-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [74]:
cols_before = jan23.shape[1]

In [75]:
from sklearn.feature_extraction import DictVectorizer

In [76]:
dv = DictVectorizer()

In [77]:
jan23["PULocationID"] = jan23["PULocationID"].astype(str)
jan23["DOLocationID"] = jan23["DOLocationID"].astype(str)

In [78]:
X_train = dv.fit_transform(jan23[["PULocationID", "DOLocationID"]].to_dict(orient="records"))

In [79]:
X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6018352 stored elements and shape (3009176, 515)>

#### Q5: training a model

Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters, where duration is the response variable
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [80]:
from sklearn.linear_model import LinearRegression

In [81]:
lr = LinearRegression()
y_train = jan23["duration"]

In [82]:
lr.fit(X_train, y_train)

LinearRegression()

In [83]:
from sklearn.metrics import root_mean_squared_error

In [84]:
root_mean_squared_error(y_true=y_train, y_pred=lr.predict(X_train))

7.649261240892796

#### Q6: evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [85]:
# calculate duration
feb23["duration"] = feb23["tpep_dropoff_datetime"] - feb23["tpep_pickup_datetime"]
feb23["duration"] = feb23["duration"].dt.seconds / 60.0

In [86]:
# clear outliers
feb23 = feb23[(feb23["duration"] >= 1) & (feb23["duration"] <= 60)]

In [87]:
feb23["PULocationID"] = feb23["PULocationID"].astype(str)
feb23["DOLocationID"] = feb23["DOLocationID"].astype(str)

In [88]:
X_valid = dv.transform(feb23[["PULocationID", "DOLocationID"]].to_dict(orient="records"))

In [89]:
y_valid = feb23["duration"]

In [90]:
root_mean_squared_error(y_true=y_valid, y_pred=lr.predict(X_valid))

7.811816659557822